In [36]:
import pandas as pd

# File path to all ages and recent grads CSV file
all_ages_file_path = "/Users/zeming/DSA3101/all-ages_cleaned.csv"
recent_grads_file_path = "/Users/zeming/DSA3101/recent-grads_cleaned.csv"

# Read the CSV files into Dataframes
all_ages_df = pd.read_csv(all_ages_file_path)
recent_grads_df = pd.read_csv(recent_grads_file_path)

# List of columns to check and drop
columns_to_drop = ["Employed_full_time_year_round", "Employed", "Unemployed"]

# Dropping the column "Employed_full_time_year_round" if it exists as it is not needed 
# Dropping "Employed" and "Unemployed" columns too as the numbers dont add up to the ratio
for column in columns_to_drop:
    if column in all_ages_df.columns:
        all_ages_df = all_ages_df.drop(columns=[column])
        print(f"Column '{column}' dropped.")
    else:
        print(f"Column '{column}' not found.")

# Perform a left join between all_ages_df and recent_grads_df on a common key (assumed to be "Major_code" 
if "ShareWomen" not in all_ages_df.columns:
    all_ages_df = all_ages_df.merge(
        recent_grads_df[['Major', 'ShareWomen']],
        on='Major',
        how='left'
    )
    print("Join completed, 'ShareWomen' column added.")
else:
    print("Join skipped, 'ShareWomen' column already exists.")

# Find and fill row and column with missing values with the mean values of the column 
def find_and_fill_missing_values(df, default_values={}):
    # Create an empty list to store the row and column information for missing values
    missing_values = []

    # Iterate over the DataFrame to check for missing values
    for row_idx, row in df.iterrows():
        for col in df.columns:
            if pd.isnull(row[col]):  # Check if the cell is missing (NaN)
                missing_values.append((row_idx, col))

                # Check if the column has a specified default value
                if col in default_values:
                    # Fill missing value with the specified default value
                    df.loc[row_idx, col] = default_values[col]
                    print(f"Filled missing value in row {row_idx}, column '{col}' with default: {default_values[col]}")
                
                # Fill missing numerical columns with the mean
                elif pd.api.types.is_numeric_dtype(df[col]):
                    mean_value = df[col].mean()
                    df.loc[row_idx, col] = mean_value
                    print(f"Filled missing value in row {row_idx}, column '{col}' with mean: {mean_value}")

    # If missing values were found, print them
    if missing_values:
        for row, col in missing_values:
            print(f"Missing value found at Row: {row}, Column: {col}")
    else:
        print("No missing values found.")
    
    # Return the updated DataFrame
    return df
    
all_ages_df = find_and_fill_missing_values(all_ages_df)

# Display the first few rows of the merged DataFrame
print(all_ages_df.head())

# Save the modified DataFrame back to the same CSV file (overwrite)
all_ages_df.to_csv(file_path, index=False)

Column 'Employed_full_time_year_round' not found.
Column 'Employed' not found.
Column 'Unemployed' not found.
Join skipped, 'ShareWomen' column already exists.
Filled missing value in row 4, column 'ShareWomen' with mean: 0.5222233649593023
Missing value found at Row: 4, Column: ShareWomen
   Major_code                                  Major  \
0        1100                    GENERAL AGRICULTURE   
1        1101  AGRICULTURE PRODUCTION AND MANAGEMENT   
2        1102                 AGRICULTURAL ECONOMICS   
3        1103                        ANIMAL SCIENCES   
4        1104                           FOOD SCIENCE   

                    Major_category   Total  Unemployment_rate  Median  P25th  \
0  Agriculture & Natural Resources  128148           0.026147   50000  34000   
1  Agriculture & Natural Resources   95326           0.028636   54000  36000   
2  Agriculture & Natural Resources   33955           0.030248   63000  40000   
3  Agriculture & Natural Resources  103549          